# 快速入门

本案例将贯穿Mindcv分类套件的基本流程，以DenseNet网络模型(加载ImageNet数据集的参数)为例子，实现对Cifar10数据集的参数微调。


## 环境准备

使用`mindspore.context.set_context()`接口对基本环境进行设置。

In [49]:
import mindspore as ms

ms.context.set_context(mode=ms.context.GRAPH_MODE, device_target='CPU')

参数说明:

- mode: 表示在GRAPH_MODE(0)或PYNATIVE_MODE(1)模式中的运行。默认值：GRAPH_MODE(0)。

- device_target: 表示待运行的目标设备，支持’Ascend’、’GPU’和’CPU’。如果未设置此参数，则使用MindSpore包对应的后端设备。

## 数据准备

使用`mindcv.data.create_dataset`接口下载和解压Cifar10数据集，将数据集转换为MindRecord格式。

In [50]:
from mindcv.data import create_dataset, create_transforms, create_loader

# 创建数据集。
cifar10_dir = './' # 你的数据存放路径
num_classes = 10 # 分类的类别数
num_workers = 8 # 读取数据的工作线程数 

dataset_train = create_dataset(name='cifar10', root=cifar10_dir, split='train', shuffle=True, num_parallel_workers=num_workers, download=False)

参数说明:

- name: 数据集名称

- dataset_dir: 包含数据集文件的根目录路径。

- split: 读取数据集的训练集（"train"）或验证集（"val"）。默认值："train"。

- shuffle: 是否混洗数据集。默认值：None。

- num_parallel_workers: 指定读取数据的工作线程数。默认值：None。

- download: 是否下载数据集。默认值：False。

使用`mindcv.data.create_transformer`接口设置需要对数据进行的数据增强操作。

In [51]:
# 创建所需的数据增强操作的列表。
trans = create_transforms(dataset_name='cifar10', image_resize=224)

参数说明:

- name: 数据集名称

- dataset_dir: 包含数据集文件的根目录路径。

- split: 读取数据集的训练集（"train"）或验证集（"val"）。默认值："train"。

- shuffle: 是否混洗数据集。默认值：None。

- num_parallel_workers: 指定读取数据的工作线程数。默认值：None。

- download: 是否下载数据集。默认值：False。

使用`mindcv.data.create_loader`接口生成用于相应任务的数据集，执行所需的数据处理（数据增强，设置batch_size等）。

In [55]:
# 执行数据增强操作，生成所需数据集。
loader_train = create_loader(dataset=dataset_train,
                             batch_size=64,
                             is_training=True,
                             num_classes=num_classes,
                             transform=trans,
                             num_parallel_workers=num_workers)

steps_per_epoch = loader_train.get_dataset_size()
print(steps_per_epoch)

782


参数说明:

- dataset: 通过标准数据集接口（mindspore.dataset.Cifar10Dataset，mindspore.dataset.CocoDataset）或者自定义数据集接口（mindspore.dataset.GeneratorDataset）加载过的数据集。

- batch_size: 指定每个批处理数据包含的数据条目。

- is_training: 读取数据集的训练集（True）或验证集（False）。默认值：False。

- num_classes: 分类的类别数。默认值：1000。
    
- transform: 所需的数据增强操作的列表。默认值：None。

- num_parallel_workers: 指定读取数据的工作线程数。默认值：None。


## 模型微调

使用`mindcv.models.create_model`接口实例化DenseNet，并加载预训练权重densenet_121_imagenet2012.ckpt（ImageNet数据集训练得到）。

> 由于Cifar10和ImageNet数据集所需分类的类别数量不同，分类器参数无法共享，所以会出现分类器参数无法加载的告警。

In [56]:
from mindcv.models import create_model

# 实例化 DenseNet-121 模型并加载预训练权重。
network = create_model(model_name='densenet121', num_classes=num_classes, pretrained=True)

[WARNING] ME(28680:11192,MainProcess):2022-09-09-18:57:32.122.060 [mindspore\train\serialization.py:709] For 'load_param_into_net', 2 parameters in the 'net' are not loaded, because they are not in the 'parameter_dict', please check whether the network structure is consistent when training and loading checkpoint.
[WARNING] ME(28680:11192,MainProcess):2022-09-09-18:57:32.123.121 [mindspore\train\serialization.py:714] classifier.weight is not loaded.
[WARNING] ME(28680:11192,MainProcess):2022-09-09-18:57:32.124.121 [mindspore\train\serialization.py:714] classifier.bias is not loaded.


参数说明:

- model_name: 需要加载的模型的规格的名称。

- num_classes: 分类的类别数。默认值：1000。

- pretrained: 是否加载与训练权重。默认值：False。

> [mindcv-0.0.1支持的模型和预训练权重]()

使用`mindcv.loss.create_loss`接口创建损失函数（cross_entropy loss）。

In [57]:
from mindcv.loss import create_loss

# 设置损失函数
loss = create_loss(name='CE')

参数说明:

- name: 需要加载的模型的规格的名称。

使用`mindcv.scheduler.create_scheduler`接口设置学习率策略（warmup_consine_decay）。

In [37]:
from mindcv.scheduler import create_scheduler

# 设置学习率策略
lr_scheduler = create_scheduler(steps_per_epoch=steps_per_epoch,
                                scheduler='constant',
                                lr=0.0001)

参数说明:

- steps_pre_epoch: 完成一轮训练所需要的步数。

- scheduler: 学习率策略的名称。

- lr: 学习率的最大值。

- min_lr: 学习率的最小值。

使用`mindcv.optim.create_optimizer`接口创建优化器。

In [38]:
from mindcv.optim import create_optimizer

# 设置优化器
opt = create_optimizer(network.trainable_params(), opt='adam', lr=lr_scheduler) 

参数说明:

- params: 需要优化的参数的列表。

- scheduler: 学习率策略的名称。

- lr: 学习率的最大值。

- min_lr: 学习率的最小值。

使用[`mindspore.Model`](https://mindspore.cn/docs/zh-CN/r1.8/api_python/mindspore/mindspore.Model.html)接口根据用户传入的参数封装可训练的实例。

In [39]:
from mindspore import Model

# 封装可训练或推理的实例
model = Model(network, loss_fn=loss, optimizer=opt, metrics={'acc'})

使用[`mindspore.LossMonitor`](https://mindspore.cn/docs/zh-CN/r1.8/api_python/mindspore/mindspore.LossMonitor.html)接口训练场景下，监控训练的loss；使用[`mindspore.TimeMonitor`](https://mindspore.cn/docs/zh-CN/r1.8/api_python/mindspore/mindspore.TimeMonitor.html)接口监控训练或推理的时间；使用[`mindspore.CheckpointConfig`](https://mindspore.cn/docs/zh-CN/r1.8/api_python/mindspore/mindspore.CheckpointConfig.html)接口保存checkpoint时的配置策略。和[`mindspore.ModelCheckpoint`](https://mindspore.cn/docs/zh-CN/r1.8/api_python/mindspore/mindspore.ModelCheckpoint.html)接口在训练过程中调用该方法可以保存网络参数。

In [40]:
from mindspore import LossMonitor, TimeMonitor, CheckpointConfig, ModelCheckpoint

# 设置监控训练的loss和接口训练时间的回调函数
loss_cb, time_cb = LossMonitor(per_print_times=10), TimeMonitor(data_size=10)

# 设置在训练过程中保存网络参数的回调函数
ckpt_save_dir = './ckpt' # 参数文件保存路径
ckpt_config = CheckpointConfig(save_checkpoint_steps=steps_per_epoch)
ckpt_cb = ModelCheckpoint(prefix='densenet121-cifar10',
                          directory=ckpt_save_dir,
                          config=ckpt_config)

# 将所有回调函数添加到一个列表中
callbacks = [loss_cb, time_cb, ckpt_cb]

使用[`mindspore.Model.train`](https://mindspore.cn/docs/zh-CN/r1.8/api_python/mindspore/mindspore.Model.html#mindspore.Model.train)接口进行模型训练。

In [ ]:
model.train(10, loader_train, callbacks=callbacks, dataset_sink_mode=False)

```text
epoch: 1 step: 10, loss is 5.531197547912598
epoch: 1 step: 20, loss is 3.432859182357788
epoch: 1 step: 30, loss is 3.01476788520813
epoch: 1 step: 40, loss is 2.79731822013855
epoch: 1 step: 50, loss is 2.3083176612854004
epoch: 1 step: 60, loss is 2.6279938220977783
epoch: 1 step: 70, loss is 2.6907918453216553
epoch: 1 step: 80, loss is 2.5941648483276367
epoch: 1 step: 90, loss is 2.298828125
epoch: 1 step: 100, loss is 2.2227189540863037
...
```

## 模型验证

加载验证数据集。

In [42]:
# 加载验证数据集
dataset_val = create_dataset(name='cifar10', root=cifar10_dir, split='test', shuffle=True, num_parallel_workers=num_workers, download=False)

# 执行数据增强操作，生成所需数据集。
loader_val = create_loader(dataset=dataset_val,
                           batch_size=64,
                           is_training=False,
                           num_classes=num_classes,
                           transform=trans,
                           num_parallel_workers=num_workers)

加载微调后的参数文件（densenet-cifar10-10_782.ckpt）到模型。

In [45]:
from mindspore import load_param_into_net, load_checkpoint

# 实例化 DenseNet-121 模型
network_eval = create_model(model_name='densenet121', num_classes=num_classes, pretrained=False)

# 加载参数到 DenseNet-121 模型中
print(load_param_into_net(network_eval, load_checkpoint('./ckpt/densenet121-cifar10-10_782.ckpt')))

[]


根据用户传入的参数封装可推理的实例，加载验证数据集，验证微调的 DenseNet121模型精度。

In [46]:
# 根据用户传入的参数封装可推理的实例
model_val =  Model(network, loss_fn=loss, optimizer=None, metrics={'acc'})

# 验证微调后的DenseNet-121的精度
acc = model_val.eval(loader_val, dataset_sink_mode=False)
print(acc)

{'acc': 0.9565}
